# SWaT Data Preprocessing Walkthrough
This notebook interactively runs the data preprocessing pipeline, step by step, with explanations and debug outputs.

In [1]:
import sys
sys.path.append('../src')  # So we can import from src/ directory

from data_preprocessing import (
    load_data, clean_data, parse_timestamps,
    select_p1_p4_columns, interpolate_missing_values, standard_scale, convert_to_numeric
)
import pandas as pd

## 1. Load Data

In [2]:
normal, attack = load_data('../data/raw/SWaT_Dataset_Normal_v1.csv', '../data/raw/SWaT_Dataset_Attack_v0.csv')
print('Normal data shape:', normal.shape)
print('Attack data shape:', attack.shape)
display(normal.head())
display(attack.head())

/Users/tim/myhub.com/playground/infra-triage/notebooks/../src/data_preprocessing.py:5: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  normal = pd.read_csv(normal_path)
/Users/tim/myhub.com/playground/infra-triage/notebooks/../src/data_preprocessing.py:6: DtypeWarning: Columns (1,9,28,46) have mixed types. Specify dtype option on import or set low_memory=False.
  attack = pd.read_csv(attack_path, sep=';')


Normal data shape: (495000, 53)
Attack data shape: (449919, 53)


,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
0,22/12/2015 4:30:00 PM,0,"124,3135",1,1,1,"251,9226","8,313446","312,7916",0,...,1,1,"9,100231",0,"3,3485","0,0002563035",1,1,1,Normal
1,22/12/2015 4:30:01 PM,0,"124,392",1,1,1,"251,9226","8,313446","312,7916",0,...,1,1,"9,100231",0,"3,3485","0,0002563035",1,1,1,Normal
2,22/12/2015 4:30:02 PM,0,"124,4705",1,1,1,"251,9226","8,313446","312,7916",0,...,1,1,"9,100231",0,"3,3485","0,0002563035",1,1,1,Normal
3,22/12/2015 4:30:03 PM,0,"124,6668",1,1,1,"251,9226","8,313446","312,7916",0,...,1,1,"9,100231",0,"3,3485","0,0002563035",1,1,1,Normal
4,22/12/2015 4:30:04 PM,0,"124,5098",1,1,1,"251,9226","8,313446","312,7916",0,...,1,1,"9,100231",0,"3,3485","0,0002563035",1,1,1,Normal


,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
0,28/12/2015 10:00:00 AM,"2,427057","522,8467",2,2,1,"262,0161","8,396437","328,6337","2,445391",...,2,1,"250,8652","1,649953","189,5988","0,000128152",1,1,1,Normal
1,28/12/2015 10:00:01 AM,"2,446274","522,886",2,2,1,"262,0161","8,396437","328,6337","2,445391",...,2,1,"250,8652","1,649953","189,6789","0,000128152",1,1,1,Normal
2,28/12/2015 10:00:02 AM,"2,489191","522,8467",2,2,1,"262,0161","8,394514","328,6337","2,442316",...,2,1,"250,8812","1,649953","189,6789","0,000128152",1,1,1,Normal
3,28/12/2015 10:00:03 AM,"2,53435","522,9645",2,2,1,"262,0161","8,394514","328,6337","2,442316",...,2,1,"250,8812","1,649953","189,6148","0,000128152",1,1,1,Normal
4,28/12/2015 10:00:04 AM,"2,56926","523,4748",2,2,1,"262,0161","8,394514","328,6337","2,443085",...,2,1,"250,8812","1,649953","189,5027","0,000128152",1,1,1,Normal


## 2. Clean Data (Drop stray first row)

## 3. Parse Timestamps and Set Index

In [3]:
print(normal.columns.tolist())
print(attack.columns.tolist())

['Timestamp', 'FIT101', 'LIT101', 'MV101', 'P101', 'P102', 'AIT201', 'AIT202', 'AIT203', 'FIT201', 'MV201', 'P201', 'P202', 'P203', 'P204', 'P205', 'P206', 'DPIT301', 'FIT301', 'LIT301', 'MV301', 'MV302', 'MV303', 'MV304', 'P301', 'P302', 'AIT401', 'AIT402', 'FIT401', 'LIT401', 'P401', 'P402', 'P403', 'P404', 'UV401', 'AIT501', 'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504', 'P501', 'P502', 'PIT501', 'PIT502', 'PIT503', 'FIT601', 'P601', 'P602', 'P603', 'Normal/Attack']
['Timestamp', 'FIT101', 'LIT101', 'MV101', 'P101', 'P102', 'AIT201', 'AIT202', 'AIT203', 'FIT201', 'MV201', 'P201', 'P202', 'P203', 'P204', 'P205', 'P206', 'DPIT301', 'FIT301', 'LIT301', 'MV301', 'MV302', 'MV303', 'MV304', 'P301', 'P302', 'AIT401', 'AIT402', 'FIT401', 'LIT401', 'P401', 'P402', 'P403', 'P404', 'UV401', 'AIT501', 'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504', 'P501', 'P502', 'PIT501', 'PIT502', 'PIT503', 'FIT601', 'P601', 'P602', 'P603', 'Normal/Attack']


In [4]:

normal = parse_timestamps(normal)
attack = parse_timestamps(attack)
print('Index type:', type(normal.index))
display(normal.head())

/Users/tim/myhub.com/playground/infra-triage/notebooks/../src/data_preprocessing.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Timestamp'] = pd.to_datetime(df['Timestamp'], dayfirst=True, errors='coerce')


Index type: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>


,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
Timestamp,,,,,,,,,,,,,,,,,,,,,
2015-12-22 16:30:00,0,"124,3135",1,1,1,"251,9226","8,313446","312,7916",0,1,...,1,1,"9,100231",0,"3,3485","0,0002563035",1,1,1,Normal
2015-12-22 16:30:01,0,"124,392",1,1,1,"251,9226","8,313446","312,7916",0,1,...,1,1,"9,100231",0,"3,3485","0,0002563035",1,1,1,Normal
2015-12-22 16:30:02,0,"124,4705",1,1,1,"251,9226","8,313446","312,7916",0,1,...,1,1,"9,100231",0,"3,3485","0,0002563035",1,1,1,Normal
2015-12-22 16:30:03,0,"124,6668",1,1,1,"251,9226","8,313446","312,7916",0,1,...,1,1,"9,100231",0,"3,3485","0,0002563035",1,1,1,Normal
2015-12-22 16:30:04,0,"124,5098",1,1,1,"251,9226","8,313446","312,7916",0,1,...,1,1,"9,100231",0,"3,3485","0,0002563035",1,1,1,Normal


In [5]:
print('Normal data shape:', normal.shape)
print('Attack data shape:', attack.shape)

Normal data shape: (495000, 52)
Attack data shape: (449919, 52)


## 4. Select P1–P4 Columns Only

In [6]:
normal, attack = select_p1_p4_columns(normal, attack)
print('Normal columns:', normal.columns.tolist())
print('Attack columns:', attack.columns.tolist())
print('Normal shape:', normal.shape)
print('Attack shape:', attack.shape)

Normal columns: ['FIT101', 'LIT101', 'MV101', 'P101', 'P102', 'AIT201', 'AIT202', 'AIT203', 'FIT201', 'MV201', 'P201', 'P202', 'P203', 'P204', 'P205', 'P206', 'DPIT301', 'FIT301', 'LIT301', 'MV301', 'MV302', 'MV303', 'MV304', 'P301', 'P302', 'AIT401', 'AIT402', 'FIT401', 'LIT401', 'P401', 'P402', 'P403', 'P404', 'UV401']
Attack columns: ['FIT101', 'LIT101', 'MV101', 'P101', 'P102', 'AIT201', 'AIT202', 'AIT203', 'FIT201', 'MV201', 'P201', 'P202', 'P203', 'P204', 'P205', 'P206', 'DPIT301', 'FIT301', 'LIT301', 'MV301', 'MV302', 'MV303', 'MV304', 'P301', 'P302', 'AIT401', 'AIT402', 'FIT401', 'LIT401', 'P401', 'P402', 'P403', 'P404', 'UV401', 'Normal/Attack']
Normal shape: (495000, 34)
Attack shape: (449919, 35)


## 5. Interpolate Missing Values (if any)

In [7]:
print('Missing values in normal:', normal.isna().sum().sum())
print('Missing values in attack:', attack.isna().sum().sum())
normal = interpolate_missing_values(normal)
attack = interpolate_missing_values(attack)
print('After interpolation:')
print('Missing values in normal:', normal.isna().sum().sum())
print('Missing values in attack:', attack.isna().sum().sum())

Missing values in normal: 0
Missing values in attack: 0
After interpolation:


/Users/tim/myhub.com/playground/infra-triage/notebooks/../src/data_preprocessing.py:28: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return df.interpolate(limit_direction='both')
/Users/tim/myhub.com/playground/infra-triage/notebooks/../src/data_preprocessing.py:28: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return df.interpolate(limit_direction='both')


Missing values in normal: 0
Missing values in attack: 0


In [8]:
# Convert to numeric (handle European decimal format)
normal = convert_to_numeric(normal)
attack = convert_to_numeric(attack)

# Standard scale
normal_z, attack_z = standard_scale(normal, attack)

## 6. Standard Scale Each Tag (Z-score)

For the last experiment on normal case; P102 std will be 0.000000: This means P102 has the same value for all rows in the normal dataset (no variation).

Why does P102 have std = 0?
This is common in industrial datasets:
- P102 might be a pump that's always OFF (value = 0) or always ON (value = 1) during normal operations
- Or it's a binary status indicator that never changes in the normal dataset

Is this a problem?
- For training: Not really. StandardScaler handles it (though the column becomes all zeros after scaling).
- For ML models: Columns with zero variance provide no information, so they're often dropped.

Options:
- Keep it as-is: The preprocessing works fine, and you can decide later whether to drop constant columns.
- Drop constant columns: Add a step to remove columns with zero variance.
- Just note it: For now, continue with your pipeline and see how it affects model training.

In [9]:
print('Scaled normal mean (should be ~0):')
display(normal_z.mean().head())
print('Scaled normal std (should be ~1):')
display(normal_z.std().head())

Scaled normal mean (should be ~0):


FIT101   -5.466155e-16
LIT101   -1.676594e-16
MV101     2.439099e-16
P101     -1.322901e-16
P102      0.000000e+00
dtype: float64

Scaled normal std (should be ~1):


FIT101    1.000001
LIT101    1.000001
MV101     1.000001
P101      1.000001
P102      0.000000
dtype: float64

In [ ]:
print('Scaled normal mean (should be ~0):')
display(attack_z.mean().head())
print('Scaled normal std (should be ~1):')
display(attack_z.std().head())

## 7. (Optional) Save Processed Data

In [10]:
normal_z.to_csv('../data/processed/normal_z.csv')
attack_z.to_csv('../data/processed/attack_z.csv')
print('Processed data saved.')

Processed data saved.
